In [266]:
import pandas as pd
import datetime as datetime
import os
# ! pip install xlsxwriter --user
from IPython.display import HTML

def df_horizontal(dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 32px">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))
    

In [267]:
data_path=r'D:\PythonDev\MyQuantFinProject\Data'
data_file='MyPort23.csv'
isPy=False 
isPriceExcel=True

# Specify Period and Timefame to caculate fund performance and

In [268]:
# If you want to see July-Oct, you have to have last trading day of June
from_str='2022-12-01'  
to_str='2023-10-31' 
timeframe='M'

pct_return_col=f"pct_change_{timeframe}"
std_col="std_of_pct_Daily"
# timeframe='W-FRI'

# Input data

In [269]:
if isPy:
    print(f"Data csv file in Path: {data_path}")
    data_file=input("CSV file name : ")

    from_str=input("From_date(2020-10-01) : ") 
    to_str=input("To_date(2022-12-31): ") 
    timeframe=input("Timeframe(W-FRI,M,Q,Y): ") 
  
    print(f"Process data from {from_str} to {to_str} in {timeframe} timeframe")
    press_y=input(f"Press y=True and n=False to run analaystics : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()


In [270]:
report_prefix_name=data_file.split('.')[0]
data_file=os.path.join(data_path,data_file)
if os.path.exists(data_file)==False:
    raise Exception(f"not found {data_file}")
else:
    print(f"Process file {data_file}")
print(data_file)

Process file D:\PythonDev\MyQuantFinProject\Data\MyPort23.csv
D:\PythonDev\MyQuantFinProject\Data\MyPort23.csv


# Read csv file and extract and transform data as need

In [271]:
df = pd.read_csv(data_file,index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df.index.set_names('date',inplace=True)
df.sort_index(inplace=True)
df=df.rename(columns={'Ticker':'symbol','close':'price'})
df=df[['symbol','price']]

#test 1
# df=df.query("symbol not in ('BTC-USD')")
# print(df.info())

df.tail(10)

,symbol,price
date,,
2023-10-30,AAXJ,62.06
2023-10-30,GLD,185.10
2023-10-30,^SET.BK,1395.85
2023-10-30,SPY,415.59
2023-10-31,AAXJ,61.41
2023-10-31,GLD,184.09
2023-10-31,IWVL.L,35.97
2023-10-31,IGM,373.89
2023-10-31,^SET.BK,1381.83


In [272]:
list_symbol=df['symbol'].unique().tolist() 

print(list_symbol)


['^SET.BK', 'AAXJ', 'IGM', 'IWVL.L', 'SPY', 'GLD']


# Basic

# Resample data and calculate Percentage of change

In [273]:
# colsROC=['symbol','date','pct_change','std_of_daily_pct','return_to_std']
colsROC=['symbol','date',pct_return_col,std_col]
df_Pct_Price=pd.DataFrame(columns=colsROC)
print(df_Pct_Price)

#https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.aggregate.html
#https://stackoverflow.com/questions/47938372/pandas-dataframe-resample-aggregate-function-use-multiple-columns-with-a-customi
#https://www.alpharithms.com/aggregating-time-series-data-with-pandas-resampling-411212/
# https://sparkbyexamples.com/pandas/pandas-aggregate-functions-with-examples/
for symbol_name in list_symbol:
    print(f"===================================={symbol_name}====================================")
    dfx_symbol=df.query('symbol==@symbol_name')
    dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
    dfx_symbol=dfx_symbol.loc[from_str:to_str]
    #print(dfx_symbol.tail())
    
    
    dfx_px=dfx_symbol.resample(timeframe).agg({'price': 'last','_pct_compare_prev_':'std'}) 
    dfx_px=dfx_px.rename(columns={'_pct_compare_prev_':std_col})

    dfx_px[pct_return_col]=dfx_px['price'].pct_change()
    dfx_px[pct_return_col]=dfx_px[pct_return_col].round(4)

    dfx_px=dfx_px.dropna() # the first  rown 
    dfx_px=dfx_px.drop(columns=['price']) # remove price becase we are focuse ROC of Price
    
    dfx_px[pct_return_col]=dfx_px[pct_return_col]*100
    dfx_px[std_col]=dfx_px[std_col]*100
    
    # dfx_px[pct_return_col]=dfx_px[pct_return_col]/dfx_px[std_col]
    
    dfx_px=dfx_px.reset_index()
    dfx_px['symbol']=symbol_name
    dfx_px=dfx_px[colsROC]
    
#     print(f"Calculate ROC of {symbol_name}")
    print(dfx_px)
    
    df_Pct_Price=pd.concat([df_Pct_Price,dfx_px],ignore_index=True,axis=0)

Empty DataFrame
Columns: [symbol, date, pct_change_M, std_of_pct_Daily]
Index: []
====================================^SET.BK====================================
    symbol       date  pct_change_M  std_of_pct_Daily
0  ^SET.BK 2023-01-31          0.17          0.467707
1  ^SET.BK 2023-02-28         -2.94          0.504113
2  ^SET.BK 2023-03-31         -0.81          1.115308
3  ^SET.BK 2023-04-30         -4.97          0.643414
4  ^SET.BK 2023-05-31          0.29          0.697295
5  ^SET.BK 2023-06-30         -1.98          0.832487
6  ^SET.BK 2023-07-31          3.52          0.677098
7  ^SET.BK 2023-08-31          0.63          0.640647
8  ^SET.BK 2023-09-30         -6.04          0.543617
9  ^SET.BK 2023-10-31         -6.09          1.009093
====================================AAXJ====================================
  symbol       date  pct_change_M  std_of_pct_Daily
0   AAXJ 2023-01-31          9.43          1.163180
1   AAXJ 2023-02-28         -7.66          1.020639
2   AAXJ 20

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_26164\317653757.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_26164\317653757.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_26164\317653757.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [274]:
print("===============List All================================")
print(df_Pct_Price)

===============List All================================
     symbol       date  pct_change_M  std_of_pct_Daily
0   ^SET.BK 2023-01-31          0.17          0.467707
1   ^SET.BK 2023-02-28         -2.94          0.504113
2   ^SET.BK 2023-03-31         -0.81          1.115308
3   ^SET.BK 2023-04-30         -4.97          0.643414
4   ^SET.BK 2023-05-31          0.29          0.697295
5   ^SET.BK 2023-06-30         -1.98          0.832487
6   ^SET.BK 2023-07-31          3.52          0.677098
7   ^SET.BK 2023-08-31          0.63          0.640647
8   ^SET.BK 2023-09-30         -6.04          0.543617
9   ^SET.BK 2023-10-31         -6.09          1.009093
10     AAXJ 2023-01-31          9.43          1.163180
11     AAXJ 2023-02-28         -7.66          1.020639
12     AAXJ 2023-03-31          3.67          1.097728
13     AAXJ 2023-04-30         -1.71          0.920696
14     AAXJ 2023-05-31         -2.85          1.038395
15     AAXJ 2023-06-30          2.52          1.052435
16     AA

# Total Period : Summarize and Combinำ return and std

In [275]:
print(f"==================Sum-Return and Avg-Std based on {timeframe}===========================")
print("Sum percentage of fund return")
dfSum_Pct_Price=df_Pct_Price.groupby(['symbol'])[[pct_return_col]].sum()
dfSum_Pct_Price.reset_index(drop=False,inplace=True)
dfSum_Pct_Price[pct_return_col]=dfSum_Pct_Price[pct_return_col].round(1)


print("==============================================================")
print("Average Std of fund ")
dfAvg_Std_Price=df_Pct_Price.groupby(['symbol'])[[std_col]].mean()
dfAvg_Std_Price.reset_index(drop=False,inplace=True)
dfAvg_Std_Price[std_col]=dfAvg_Std_Price[std_col].round(2)

dfValueSummary=pd.merge(left=dfSum_Pct_Price,right=dfAvg_Std_Price,on='symbol')
dfValueSummary.fillna(value=0,inplace=True)
dfValueSummary.set_index('symbol',inplace=True)
dfValueSummary

==================Sum-Return and Avg-Std based on M===========================
Sum percentage of fund return
Average Std of fund 


,pct_change_M,std_of_pct_Daily
symbol,,
AAXJ,-3.9,1.03
GLD,9.2,0.80
IGM,31.0,1.31
IWVL.L,6.1,0.83
SPY,9.6,0.83
^SET.BK,-18.2,0.71


# Timeframe Period 

# Pivot Percent Return & Std and theire Rank To Dataframe

In [276]:
print("==============================================================")
print(f"Pivot Percent Return and Rank {timeframe} To Dataframe")
dfPct=df_Pct_Price.pivot(index='date', columns='symbol', values=pct_return_col)
dfRank=dfPct[dfPct.columns.tolist()].rank(axis=1)
# print(dfPct.info())

Pivot Percent Return and Rank M To Dataframe


In [277]:
print("==============================================================")
print(f"Pivot Daily STD and Rank in {timeframe} and Rank To Dataframe")
dfStd=df_Pct_Price.pivot(index='date', columns='symbol', values=std_col)
dfRankStd=dfStd[dfStd.columns.tolist()].rank(axis=1,ascending=False)


Pivot Daily STD and Rank in M and Rank To Dataframe


In [278]:
df_horizontal([dfRank,dfPct])
df_horizontal([dfRankStd,dfStd])

symbol,AAXJ,GLD,IGM,IWVL.L,SPY,^SET.BK
date,,,,,,
2023-01-31,5.0,2.0,6.0,4.0,3.0,1.0
2023-02-28,1.0,2.0,6.0,5.0,4.0,3.0
2023-03-31,4.0,5.0,6.0,2.0,3.0,1.0
2023-04-30,2.0,4.0,3.0,5.0,6.0,1.0
2023-05-31,2.0,3.0,6.0,1.0,5.0,4.0
2023-06-30,3.0,1.0,4.0,6.0,5.0,2.0
2023-07-31,6.0,1.0,5.0,4.0,2.0,3.0
2023-08-31,1.0,4.0,5.0,2.0,3.0,6.0
2023-09-30,5.0,4.0,1.0,6.0,3.0,2.0


symbol,AAXJ,GLD,IGM,IWVL.L,SPY,^SET.BK
date,,,,,,
2023-01-31,2.0,5.0,1.0,4.0,3.0,6.0
2023-02-28,2.0,4.0,1.0,5.0,3.0,6.0
2023-03-31,6.0,3.0,1.0,2.0,4.0,5.0
2023-04-30,2.0,3.0,1.0,6.0,4.0,5.0
2023-05-31,2.0,4.0,1.0,3.0,5.0,6.0
2023-06-30,2.0,5.0,1.0,4.0,6.0,3.0
2023-07-31,1.0,5.0,2.0,3.0,6.0,4.0
2023-08-31,2.0,6.0,1.0,4.0,3.0,5.0
2023-09-30,2.0,6.0,1.0,4.0,3.0,5.0


# Combine Return and Std Rank as Score

In [279]:
print("==============================================================")
print("Combine Both Pct Return and Rank")
print("Return :higher total score , greater performance")
print("Std : higher total score , worse perfromance")
dfRankSummary=pd.concat([
                     dfRank.sum().to_frame(name='sum_return_rank'),
                     dfRankStd.sum().to_frame(name='sum_std_rank'),  
                    ],axis=1)
dfRankSummary.fillna(value=0,inplace=True)
dfRankSummary

Combine Both Pct Return and Rank
Return :higher total score , greater performance
Std : higher total score , worse perfromance


,sum_return_rank,sum_std_rank
symbol,,
AAXJ,32.0,24.0
GLD,32.0,45.0
IGM,46.5,11.0
IWVL.L,37.0,40.0
SPY,38.5,43.0
^SET.BK,24.0,47.0


# Convert value score   and rank score to final rank

In [280]:
dfValueSummary['value_return_rank']=dfValueSummary[pct_return_col].rank()
dfValueSummary['value_std_rank']=dfValueSummary[std_col].rank(ascending=False)

In [281]:
# real score
dfRankSummary['return_rank']=dfRankSummary['sum_return_rank'].rank()
dfRankSummary['std_rank']=dfRankSummary['sum_std_rank'].rank()



In [282]:
df_horizontal([dfValueSummary, dfRankSummary])

,pct_change_M,std_of_pct_Daily,value_return_rank,value_std_rank
symbol,,,,
AAXJ,-3.9,1.03,2.0,2.0
GLD,9.2,0.80,4.0,5.0
IGM,31.0,1.31,6.0,1.0
IWVL.L,6.1,0.83,3.0,3.5
SPY,9.6,0.83,5.0,3.5
^SET.BK,-18.2,0.71,1.0,6.0
,sum_return_rank,sum_std_rank,return_rank,std_rank
symbol,,,,
AAXJ,32.0,24.0,2.5,2.0


# Summarize Total Score 

In [283]:
dfFinalScore= pd.concat([ dfValueSummary[['value_return_rank','value_std_rank']],dfRankSummary[['return_rank','std_rank']]],axis=1)
dfFinalScore['TotalScore']=dfFinalScore.sum(axis=1)
dfFinalScore=dfFinalScore.sort_values(by='TotalScore',ascending=False)

dfFinalScore['pctChangeScore']=dfFinalScore['TotalScore'].pct_change()*100
dfFinalScore['pctChangeScore']=dfFinalScore['pctChangeScore'].round(0)
dfFinalScore.fillna(value=0,inplace=True)

dfFinalScore=dfFinalScore[['TotalScore','pctChangeScore','value_return_rank','value_std_rank','return_rank','std_rank']]
dfFinalScore

,TotalScore,pctChangeScore,value_return_rank,value_std_rank,return_rank,std_rank
symbol,,,,,,
SPY,17.5,0.0,5.0,3.5,5.0,4.0
GLD,16.5,-6.0,4.0,5.0,2.5,5.0
IGM,14.0,-15.0,6.0,1.0,6.0,1.0
^SET.BK,14.0,0.0,1.0,6.0,1.0,6.0
IWVL.L,13.5,-4.0,3.0,3.5,4.0,3.0
AAXJ,8.5,-37.0,2.0,2.0,2.5,2.0


In [284]:
if isPriceExcel:
    print("Export outcoome as Excel")
    with pd.ExcelWriter(f"{report_prefix_name}_{timeframe}_{from_str}To{to_str}.xlsx") as writer:
        df_Pct_Price.to_excel(writer, sheet_name=f"{timeframe}")
        
        dfPct.to_excel(writer, sheet_name=f"{timeframe}-Return")
        dfRankStd.to_excel(writer, sheet_name=f"{timeframe}-RankStd")
        
        dfStd.to_excel(writer, sheet_name=f"{timeframe}-Std")
        dfRank.to_excel(writer, sheet_name=f"{timeframe}-RankReturn")
        
        
        dfValueSummary.to_excel(writer, sheet_name="Value-Summary")
        dfRankSummary.to_excel(writer, sheet_name="Rank-Summary")

        dfFinalScore.to_excel(writer, sheet_name="FinalScore")

Export outcoome as Excel
